In [ ]:
%pip install hydrotools hydroeval progressbar xgboost dataretrieval streamstats earthengine-api geopandas matplotlib boto3 openpyxl tqdm


# 1. Preliminary preparations

## 1.1. Import packages

In [1]:
# Bsic packages
from progressbar import ProgressBar
from tqdm import tqdm
from zipfile import ZipFile
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

# Hydrological Packages
from scipy import optimize
#import ee
import ulmo
import streamstats
import geopandas as gpd
from shapely.geometry import Polygon, Point

# System package
import platform
import os
import time

# My packages
from g_data_get import Retriever
import g_Get_StreamStats
#import EE_funcs

# AWS packages
import boto3

## 1.2. Find the directory based on the OS system

In this function we find the address to the directory that we want to use. 

In [2]:
if platform.system() == 'Windows':
    onedrive_path = 'E:/OneDrive/OneDrive - The University of Alabama/10.material/01.data/usgs_data/'
    box_path = 'C:/Users/snaserneisary/Box/NWM-ML/'

elif platform.system() == 'Darwin':
    onedrive_path = '/Users/savalan/Library/CloudStorage/OneDrive-TheUniversityofAlabama/02.projects/03.ciroh/04.data/'
    box_path = '/Users/savalan/Library/CloudStorage/Box-Box/NWM-ML/Data/NWM/ut/'
    
elif platform.system() == 'Linux':
    path_01 = '/home/snaserneisary/01.projects/01.ciroh_p8/NWM-ML/Savalan/'

# 2. Data preparation

## 2.1. Set the parameters 

Here we set parameters like the start and end dates, and station numbers.

In [3]:
start = '1980-01-01'  # Start date.
end = '2020-12-31'  # End date.
end_year = 2020  # Prefered end date. 
missing_data = 10  # Prefered missing data value. 
time_period = 11  # Prefered number of years. 

In [4]:
# Read the GSL station list. 
states_list = pd.read_excel(path_01 + '02.input/final_stations_p8.xlsx', sheet_name='all')

# Find the valid stations. 
accpeted_stations = Retriever.valid_station(missing_data, time_period, end_year, states_list)

In [5]:
accpeted_stations['station'] = accpeted_stations['station'].astype(str)  # Change the column dtype. 
NWIS_NHDPlus = Retriever.get_nhd_model_info(accpeted_stations)[1]  # Get NWM reaches for each USGS station
NWIS_NHDPlus.dropna(inplace=True)  # Drop the stations that don't have NWM reaches. 
NWIS_NHDPlus.reset_index(drop=True, inplace=True)
nwis_sites = NWIS_NHDPlus.NWISid.tolist()  # Convert the NWIS site list to a Python list. 
len(nwis_sites)

Getting NHD reaches


40

In [6]:
# Start the S3 bucket access. 
from botocore.client import Config
from botocore import UNSIGNED
#load access key
home = os.path.expanduser('~')
keypath = "aws_key.csv"
access = pd.read_csv(f"{home}/{keypath}")

#start session
session = boto3.Session(
    aws_access_key_id=access['Access key ID'][0],
    aws_secret_access_key=access['Secret access key'][0])
bucket_name = 'streamflow-app-data'
s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
bucket = s3.Bucket(bucket_name)

## 2.2. Get various datasets

#### 2.2.1. Get USGS station info

In [7]:
start_time = time.time()

# Ask the user for the data source
ask = input('Which data source [local/online]?')
if ask == 'online':
    # Get StreamStats information
    StreamStats, coordinates_list = Get_StreamStats.get_USGS_site_info(nwis_sites)

    # Modify data
    StreamStats['station_id'] = StreamStats['station_id'].astype('str')

    # Save data
    StreamStats.to_csv(path_01 + '03.output/01.data_preparation/stream_stats.csv')
    with open(path_01 + '03.output/01.data_preparation/coordinates.pickle', 'wb') as handle:
        pickle.dump(coordinates_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
elif ask == 'local':
    StreamStats = pd.read_csv(path_01 + '03.output/01.data_preparation/stream_stats.csv')
    StreamStats.pop('Unnamed: 0') # Drop the residual column created after reading the local file. 
    # Modify data
    StreamStats['station_id'] = StreamStats['station_id'].astype(int).astype(str)
    with open(path_01 + '03.output/01.data_preparation/coordinates.pickle', 'rb') as handle:
        coordinates_list = pickle.load(handle)
    
print("Run Time:" + " %s seconds " % (time.time() - start_time))

Which data source [local/online]? local


Run Time: 4.326514959335327 seconds 


In [8]:
# Drop the rows that we don't have the data for them.
nwis_sites = pd.DataFrame(nwis_sites)
nwis_sites = nwis_sites.loc[~StreamStats.isna().any(axis=1)].reset_index(drop=True)
NWIS_NHDPlus = NWIS_NHDPlus.loc[~StreamStats.isna().any(axis=1)].reset_index(drop=True)
StreamStats = StreamStats.loc[~StreamStats.isna().any(axis=1)].reset_index(drop=True)

#### 2.2.2. Get USGS data

In [9]:
start_time = time.time()
# Create variables
flow = {}
all_sites_flow = pd.DataFrame()
pbar = ProgressBar()
# Run the code for each station
for site in tqdm(nwis_sites.values.flatten().tolist()):
    
    #print('Getting streamflow data for ', site, end='\r', flush=True)
    flow[site] = Retriever.get_usgs(site, start, end)

    #make the date the index for plotting and resampling
    flow[site]['datetime'] = flow[site]['value_time']
    flow[site].set_index('datetime', inplace = True)

    #clean the data
    flow[site] = flow[site][flow[site]['value'] > 0]

    #resample to a daily resolution
    flow[site] = pd.DataFrame(flow[site]['value']).rename(columns = {'value':'flow_cfs'})
    flow[site] = flow[site].resample('D').mean()
    flow[site]['station_id'] = site
    all_sites_flow = pd.concat([all_sites_flow, flow[site]])
all_sites_flow.reset_index(inplace=True)
    
print("Run Time:" + " %s seconds " % (time.time() - start_time))

100%|██████████| 27/27 [12:50<00:00, 28.54s/it]

Run Time: 770.6466863155365 seconds 


#### 2.2.3. Get storage data. 

In [10]:
start_time = time.time()
storage_info = pd.read_csv(path_01 + '02.input/storage_info.csv', nrows=12)  # Storage list and their information. 
storage_data = pd.read_csv(path_01 + '02.input/storage_data.csv')  # Daily data list for all the storages. 
storage_data['Date'] = pd.to_datetime(storage_data['Date'])  # Convert the type of a column.. 

# Create the variables.
all_storage = pd.DataFrame()

for i in tqdm(range(len(StreamStats))):
    #print(StreamStats.iloc[i, 0], end="\033[K")
    
    # Get each station's watershed coordinates. 
    #watershed = streamstats.watershed.Watershed(StreamStats.iloc[i, 1], StreamStats.iloc[i, 2])
    #watershed_boundary = watershed.boundary['features'][0]['geometry']['coordinates']

    # Given list of coordinates
    #coordinates = watershed_boundary
    coordinates = coordinates_list[StreamStats.iloc[i, 0]]

    # Create Polygon geometry from the coordinates
    geometry = [Polygon(coord) for coord in coordinates]

    # Create a GeoDataFrame with the Polygon geometry
    gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")

    # Create a GeoDataFrame with storage coordiates. 
    geometry_points = [Point(xy) for xy in zip(storage_info['lon'], storage_info['lat'])]
    points_gdf = gpd.GeoDataFrame(geometry=geometry_points, crs='EPSG:4326')

    # Search to see which storages are in the watershed. 
    snotel_station_list = storage_info[points_gdf.within(gdf.geometry.iloc[0])]
    
    # Create variables
    all_snotel_swe = pd.DataFrame()

    
    # Check the number of storages for each station. 
    if len(snotel_station_list) > 1:
        for station_index in range(len(snotel_station_list)):
            sitecode = snotel_station_list.iloc[station_index, 0]  # Get the storage name. 
            start_date = pd.to_datetime(start)  # Get the start date.
            end_date = pd.to_datetime(end)  # Get the end date. 

            # Read the specific storage data, and prepare it for the next steps. 
            snotel_swe = storage_data[(storage_data['storage_name'] == sitecode) & (storage_data['Date'] >= start) & (storage_data['Date'] <= end)]
            snotel_swe.drop([snotel_swe.columns[0], 'storage_name'], inplace=True, axis=1) 
            
            # Get the percent of storage capacity which is full. 
            snotel_swe['Storage (af)'] = snotel_swe['Storage (af)'] / snotel_station_list.iloc[station_index, 4] * 100
            snotel_swe.rename(columns={'Storage (af)': sitecode}, inplace=True)

            # If it is the first snow station.
            if station_index == 0:
                all_snotel_swe = snotel_swe
                
            # If it is not the first snow station. 
            else:
                all_snotel_swe = pd.merge(all_snotel_swe, snotel_swe, on='Date')  # Merge based on date. 

        # Prepare data. 
        all_snotel_swe.set_index('Date', inplace=True)
        all_snotel_swe['sum'] = all_snotel_swe.sum(axis=1)/ len(snotel_station_list)  # Average of all stations. 
        all_snotel_swe.drop(all_snotel_swe.iloc[:, :-1], inplace=True, axis=1)
        all_snotel_swe.rename(columns={'sum': 'storage'}, inplace=True)
        all_snotel_swe['station_id'] = StreamStats.iloc[i, 0]
        
    # If there is only one storage for this station. 
    elif len(snotel_station_list) == 1:
            sitecode = snotel_station_list.iloc[0, 0]  # Get the storage name. 
            start_date = pd.to_datetime(start)  # Get the start date.
            end_date = pd.to_datetime(end)  # Get the end date. 
            
            # Read the specific storage data, and prepare it for the next steps. 
            all_snotel_swe = storage_data[(storage_data['storage_name'] == sitecode) & (storage_data['Date'] >= start) & (storage_data['Date'] <= end)]
            all_snotel_swe.drop([all_snotel_swe.columns[0], 'storage_name'], inplace=True, axis=1) 
            all_snotel_swe['Storage (af)'] = all_snotel_swe['Storage (af)'] / snotel_station_list.iloc[0, 4] * 100
            all_snotel_swe.rename(columns={'Storage (af)': 'storage'}, inplace=True)
            all_snotel_swe['station_id'] = StreamStats.iloc[i, 0]
            all_snotel_swe.set_index('Date', inplace=True)
            

    # If there is no storage for this station. 
    else:
        
        # Create an zero dataframe. 
        all_snotel_swe = pd.DataFrame({'storage': [0], 'station_id': StreamStats.iloc[i, 0]}, index=pd.date_range(start=start, end=end))

    # Add all the data for each station to the previous one and make a complete dataset.
    all_storage = pd.concat([all_storage, all_snotel_swe])
all_storage.reset_index(inplace=True) 
all_storage.rename(columns={'index': 'datetime'}, inplace=True)
all_storage['datetime'] = pd.to_datetime(all_storage['datetime'])
all_storage['station_id'] = all_storage['station_id'].astype(int).astype(str)
all_storage.to_csv(path_01 + '03.output/storage_data.csv')
print("Run Time:" + " %s seconds " % (time.time() - start_time))
    

100%|██████████| 27/27 [00:02<00:00,  9.16it/s]


Run Time: 13.723348617553711 seconds 


#### 2.2.4. Get snotel swe data.

In [13]:
start_time = time.time()
points_df = pd.read_csv(path_01 + '02.input/Snotel-CDEC-SnowObs.csv')

all_snow_swe = pd.DataFrame()
for i in tqdm(range(len(StreamStats))):
    #print(StreamStats.iloc[i, 0])
    #watershed = streamstats.watershed.Watershed(StreamStats.iloc[i, 1], StreamStats.iloc[i, 2])
    #watershed_boundary = watershed.boundary['features'][0]['geometry']['coordinates']

    # Given list of coordinates
    coordinates = coordinates_list[str(int(StreamStats.iloc[i, 0]))]

    # Create Polygon geometry from the coordinates
    geometry = [Polygon(coord) for coord in coordinates]

    # Create a GeoDataFrame with the Polygon geometry
    gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")

    geometry_points = [Point(xy) for xy in zip(points_df['longitude'], points_df['latitude'])]
    points_gdf = gpd.GeoDataFrame(geometry=geometry_points, crs='EPSG:4326')

    snotel_station_list = points_df[points_gdf.within(gdf.geometry.iloc[0])]

    all_snotel_swe = pd.DataFrame()

    if len(snotel_station_list) > 1:
        for station_index in range(len(snotel_station_list)):
            sitecode = snotel_station_list.iloc[station_index, 0]
            start_date = pd.to_datetime(start)
            end_date = pd.to_datetime(end)

            snotel_swe = Retriever.get_snotel(sitecode, start_date, end_date)
            #snotel_swe.rename(columns={snotel_swe.columns[0]: 'swe'}, inplace=True)
            snotel_swe.drop(snotel_swe.iloc[:, 1:], inplace=True, axis=1)

            if station_index == 0:

                all_snotel_swe = snotel_swe

            else:

                all_snotel_swe = pd.merge(all_snotel_swe, snotel_swe, left_index=True, right_index=True)
        all_snotel_swe['sum'] = all_snotel_swe.sum(axis=1)/ len(snotel_station_list)
        all_snotel_swe.drop(all_snotel_swe.iloc[:, :-1], inplace=True, axis=1)
        all_snotel_swe.rename(columns={'sum': 'swe'}, inplace=True)
        all_snotel_swe['station_id'] = StreamStats.iloc[i, 0]
    elif len(snotel_station_list) == 1:
            sitecode = snotel_station_list.iloc[0, 0]
            start_date = pd.to_datetime(start)
            end_date = pd.to_datetime(end)

            all_snotel_swe = Retriever.get_snotel(sitecode, start_date, end_date)
            all_snotel_swe.rename(columns={all_snotel_swe.columns[0]: 'swe'}, inplace=True)
            all_snotel_swe.drop(snotel_swe.iloc[:, 1:], inplace=True, axis=1)
            all_snotel_swe['station_id'] = StreamStats.iloc[i, 0]

    else:

        all_snotel_swe = pd.DataFrame({'swe': [0], 'station_id': StreamStats.iloc[i, 0]}, index=pd.date_range(start=start, end=end))
    all_snow_swe = pd.concat([all_snow_swe, all_snotel_swe])
all_snow_swe = all_snow_swe.iloc[:, :2]
all_snow_swe.reset_index(inplace=True)
all_snow_swe.rename(columns={'index': 'datetime'}, inplace=True)
all_snow_swe['datetime'] = pd.to_datetime(all_snow_swe['datetime'])
all_snow_swe['station_id'] = all_snow_swe['station_id'].astype(int).astype(str)
all_snow_swe.to_csv(path_01 + '03.output/snow_data.csv')
print("Run Time:" + " %s seconds " % (time.time() - start_time))


100%|██████████| 27/27 [03:22<00:00,  7.50s/it]


Run Time: 204.18125462532043 seconds 


In [59]:
# Get data for each reach
NWM_obs = Retriever.get_NWM_data(NWIS_NHDPlus, start, end, bucket)

# Merge into one DF
NWM_obs_df = pd.DataFrame()
pbar = ProgressBar()
for site in tqdm(NWM_obs):

    # This section is to make sure to remove the USGS gauges that don't have any NWM reach. 
    if isinstance(NWM_obs[site], int):
        NWM_obs[site] = pd.DataFrame({'NWM_flow': [np.nan] * 14610, 'datetime': NWM_obs['7887898']['datetime'], 'station_id': [np.nan] * 14610})
    NWM_obs_df = pd.concat([NWM_obs[site], NWM_obs_df])

# Convert to datetime to make sure it joins nicely with the clim_df
NWM_obs_df['datetime'] = pd.to_datetime(NWM_obs_df['datetime'])
NWM_obs_df.to_csv(path_01 + '03.output/nwm_flow.csv')

100%|██████████| 27/27 [00:00<00:00, 132.25it/s]


#### 2.2.3. Get temperature and precipitation data

In [ ]:
# # Activate google-earth engine

# functions

# def dict_to_df(DF, temporal_resample, variable):
#     var_df = pd.DataFrame()
#     kg_in = 0.04
#     pbar = ProgressBar()
#     for site in pbar(DF):
#         DF[site].set_index('datetime', inplace = True)

#         if variable == 'temperature':
#             #make columns for Fahrenheit
#             DF[site] = DF[site].resample(temporal_resample).max()
#             DF[site].reset_index(inplace = True)
#             DF[site]['temperature_F'] = (DF[site]['temperature']*9/5)+32
#             DF[site].pop('temperature')
#             DF[site]['station_id'] = site
#         if variable == 'total_precipitation':
#             DF[site] = DF[site].resample(temporal_resample).sum()
#             DF[site].reset_index(inplace = True)
#             #make columns for inches
#             DF[site]['precipitation_in'] = DF[site]['total_precipitation']*kg_in
#             DF[site].pop('total_precipitation')
#             DF[site]['station_id'] = site

#         var_df = pd.concat([var_df, DF[site]])

#     var_df.reset_index(inplace=True, drop=True)
#     return var_df, DF
#we can make a plotting function that takes in a list of up to 4 model predictions


# def climate():
#     #ee.Authenticate()
#     ee.Initialize()
#     import gc
#     # get temperature and precipitation data
# 
#     #NLDAS temperature
#     temp = ee.ImageCollection('NASA/NLDAS/FORA0125_H002').select('temperature').filterDate(start, end)
#     #NLDAS precipitation
#     precip = ee.ImageCollection('NASA/NLDAS/FORA0125_H002').select('total_precipitation').filterDate(start, end)
# 
#     # Define slc location of interest, lets use the USGS guage locations!
#     lon = {}
#     lat = {}
#     poi = {}
# 
#     for site in tqdm(np.arange(0,len(nwis_sites),1)):
#         lon[site] = StreamStats['Long'][site]
#         lat[site] = StreamStats['Lat'][site]
#         poi[site] = ee.Geometry.Point(lon[site], lat[site])
# 
#     scale = 1000  # scale in meters
#     temp_temp = {}
#     precip_temp = {}
#     # Get the data for the pixel intersecting the point at each station
# 
# 
#     pbar = ProgressBar()
#     for site in tqdm(np.arange(0,len(nwis_sites),1)):
#         name = StreamStats['station_id'][site]
#         temp_poi = temp.getRegion(poi[site], scale).getInfo()
#         precip_poi = precip.getRegion(poi[site], scale).getInfo()
# 
#     #Convert pointer object array to dataframe to work with
# 
# 
# 
#         name = StreamStats['station_id'][site]
#         temp_temp[name] = EE_funcs.ee_array_to_df(temp_poi,['temperature'])
#         precip_temp[name] = EE_funcs.ee_array_to_df(precip_poi,['total_precipitation'])
#         #need to add in time to precip data, same daterange as temperaure
#         precip_temp[name]['time'] = temp_temp[name]['time']
#         del temp_poi
#         del precip_poi
# 
# 
#         gc.collect()
# 
# 
#     #resample to the desired temporal scale, take the respective statistic from the data of resampling
#     temporal_resample = 'D'
# 
#     T, T_DF = dict_to_df(temp_temp, temporal_resample, 'temperature')
#     P, P_DF = dict_to_df(precip_temp, temporal_resample, 'total_precipitation')
# 
#     Clim_DF = pd.merge(T, P, on=['datetime', 'station_id'])
#     Clim_DF.pop('time_x')
#     Clim_DF.pop('time_y')
#     return Clim_DF
# climate_all = climate()

In [ ]:
# # Save the temperature and precipitation data
# climate_all['station_id'] = climate_all['station_id'].astype(int).astype(str)
# climate_all.to_csv(path_01 + '03.output/climate_data.csv')


In [56]:
seasonality = pd.read_csv(path_01 + '02.input/seasonality.csv')
# variable_list = [climate_all, all_storage, all_snow_swe, NWM_obs_df]
variable_list = [all_storage, all_snow_swe, NWM_obs_df]
final_data_set = pd.merge(StreamStats, all_sites_flow, on=['station_id'])
final_data_set['month'] = final_data_set.datetime.dt.month
final_data_set = pd.merge(final_data_set, seasonality, on=['month'])
for variable in variable_list:
    final_data_set = pd.merge(final_data_set, variable, on=['datetime', 'station_id'])
final_data_set['DOY'] = final_data_set.datetime.dt.dayofyear
final_data_set.pop('month')

#What feature are relevant. Lets remove NWIS_site_id
#final_data_set.pop('station_id')

#remove old datetime column
#final_data_set.pop('datetime')

#drop rows with nan
final_data_set.dropna(inplace = True)

# Sort the outputs of the best model based on date. 
final_data_set = final_data_set.sort_values(by=['station_id', 'datetime'])

# Save and show the output. 
final_data_set.to_csv(path_01 + '03.output/raw_training_data.csv')
final_data_set

,station_id,Lat,Long,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,Mean_Ann_Precip_in,datetime,flow_cfs,s1,s2,storage,swe,NWM_flow,DOY
0,10011500,40.965225,-110.853508,174.00,9720.0,67.70,1.2,0.120,2.94,27.2,34.8,2010-10-28,78.555206,-0.891007,-0.453991,0.0,1.2,55.0,301
1,10011500,40.965225,-110.853508,174.00,9720.0,67.70,1.2,0.120,2.94,27.2,34.8,2010-10-29,98.611458,-0.891007,-0.453991,0.0,1.2,55.0,302
2,10011500,40.965225,-110.853508,174.00,9720.0,67.70,1.2,0.120,2.94,27.2,34.8,2010-10-30,97.602081,-0.891007,-0.453991,0.0,1.1,54.0,303
3,10011500,40.965225,-110.853508,174.00,9720.0,67.70,1.2,0.120,2.94,27.2,34.8,2010-10-31,99.331253,-0.891007,-0.453991,0.0,1.2,54.0,304
17379,10011500,40.965225,-110.853508,174.00,9720.0,67.70,1.2,0.120,2.94,27.2,34.8,2010-11-01,95.763542,-0.998630,0.052336,0.0,1.2,54.0,305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206977,10172952,41.858530,-113.327219,8.57,8190.0,6.01,0.0,0.021,40.20,57.5,17.3,2020-09-26,1.195000,-0.529919,-0.848048,0.0,0.0,0.0,270
206978,10172952,41.858530,-113.327219,8.57,8190.0,6.01,0.0,0.021,40.20,57.5,17.3,2020-09-27,1.202292,-0.529919,-0.848048,0.0,0.0,0.0,271
206979,10172952,41.858530,-113.327219,8.57,8190.0,6.01,0.0,0.021,40.20,57.5,17.3,2020-09-28,1.190833,-0.529919,-0.848048,0.0,0.0,0.0,272
206980,10172952,41.858530,-113.327219,8.57,8190.0,6.01,0.0,0.021,40.20,57.5,17.3,2020-09-29,1.176250,-0.529919,-0.848048,0.0,0.0,0.0,273


In [58]:
final_data_set[final_data_set.station_id == '10011500'].iloc[:35]

,station_id,Lat,Long,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,Mean_Ann_Precip_in,datetime,flow_cfs,s1,s2,storage,swe,NWM_flow,DOY
0,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-28,78.555206,-0.891007,-0.453991,0.0,1.20,55.0,301
1,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-29,98.611458,-0.891007,-0.453991,0.0,1.20,55.0,302
2,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-30,97.602081,-0.891007,-0.453991,0.0,1.10,54.0,303
3,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-31,99.331253,-0.891007,-0.453991,0.0,1.20,54.0,304
17379,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-11-01,95.763542,-0.998630,0.052336,0.0,1.20,54.0,305
17380,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-11-02,82.590630,-0.998630,0.052336,0.0,1.15,53.0,306
17381,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-11-03,92.077080,-0.998630,0.052336,0.0,1.05,53.0,307
17382,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-11-04,90.990623,-0.998630,0.052336,0.0,0.85,53.0,308
17383,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-11-05,87.856247,-0.998630,0.052336,0.0,0.75,53.0,309
17384,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-11-06,90.870834,-0.998630,0.052336,0.0,0.70,52.0,310
